In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
import sqlite3
import numpy as np
from tqdm import tqdm
from pathlib import Path
import openslide
import time
import pickle
import cv2
from glob import glob

In [4]:
import torchvision.transforms as transforms

In [45]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *

from object_detection_fastai.helper.wsi_loader import SlideContainer
from object_detection_fastai.helper.object_detection_helper import *
from object_detection_fastai.loss.RetinaNetFocalLoss import RetinaNetFocalLoss
from object_detection_fastai.models.RetinaNet import RetinaNet
from object_detection_fastai.helper.nms_center_distance import non_max_suppression_by_distance

In [22]:
def rescale_box(bboxes, size: Tensor):
    bboxes[:, :2] = bboxes[:, :2] - bboxes[:, 2:] / 2
    bboxes[:, :2] = (bboxes[:, :2] + 1) * size / 2
    bboxes[:, 2:] = bboxes[:, 2:] * size / 2
    bboxes = bboxes.long()
    return bboxes

In [37]:
size = 1024
level = 0

files = []

In [38]:
path = Path('../Slides/')

In [39]:
list(path.glob("*/*.*"))

[WindowsPath('../Slides/Aperio/24_EIPH_576255 Berliner Blau.svs'),
 WindowsPath('../Slides/AxioScan/Z_BB_576255_1.tif'),
 WindowsPath('../Slides/NanoZoomer2.0HT/N2_BB_576255_1.ndpi'),
 WindowsPath('../Slides/NanoZoomerS210/N1_BB_576255_1.ndpi')]

In [40]:
for slide_path in tqdm(path.glob("*/*.*")):
    
    slide = openslide.open_slide(str(slide_path))
    level = level#slide.level_count - 1
    level_dimension = slide.level_dimensions[level]
    down_factor = slide.level_downsamples[level]

    files.append(SlideContainer(slide_path,[[0], [1]], level, size, size))

4it [00:00, 10.09it/s]


In [28]:
slide_path.parent.stem

'NanoZoomerS210'

In [31]:
model_file = "Equine-Fold-2.p"

device = torch.device("cuda")
model_information = torch.load(model_file, map_location='cpu') \
                    if torch.cuda.is_available() ==  False \
                    else torch.load(model_file)

model_weights = model_information['model']
anchors = model_information["anchors"]
mean = model_information["mean"]
std = model_information["std"]
shape = model_information["size"]
n_classes = model_information['n_classes']
n_anchors = model_information['n_anchors']
sizes = model_information['sizes']
chs = model_information['chs']
encoder = model_information['encoder']
n_conv = model_information['n_conv']
level = model_information['level']


encoder = create_body(models.resnet18, False, -2)

model = RetinaNet(encoder, n_classes, n_anchors=n_anchors, sizes=sizes,
                                       chs=chs, n_conv=3)
model.load_state_dict(model_weights)
model.to(device)

RetinaNet(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [32]:
detect_thresh = 0.3 
nms_thresh = 0.3

In [49]:
import torchvision.transforms as transforms

debug_level = 1
annotations = []


with torch.no_grad():
    for slide_container in tqdm(files):
        
        image_boxes = []
        
        for x in range(0, slide_container.slide.level_dimensions[level][0] - 2 * size, 750): #int(size / 2)
            for y in range(0, slide_container.slide.level_dimensions[level][1] - 2 * size, 750): #int(size / 2)
                x_real = x  # * slide_container.down_factor, \
                y_real = y  # * slide_container.down_factor

                patch_ori = slide_container.get_patch(x, y)
                patch = pil2tensor(patch_ori / 255., np.float32)

                patch = transforms.Normalize(mean, std)(patch)

                class_pred_batch, bbox_pred_batch, _ = model.eval()(
                    patch[None, :, :, :].cuda())
                for clas_pred, bbox_pred in zip(class_pred_batch, bbox_pred_batch):

                    bbox_pred, scores, preds = process_output(clas_pred, bbox_pred, anchors, detect_thresh)

                    if bbox_pred is not None:
                        to_keep = nms(bbox_pred, scores, nms_thresh)
                        bbox_pred, preds, scores = bbox_pred[to_keep].cpu(), preds[to_keep].cpu(), scores[to_keep].cpu()

                        t_sz = torch.Tensor([size, size])[None].float()

                        bbox_pred = rescale_box(bbox_pred, t_sz)

                        patch_ori = patch_ori.astype(np.uint8)
                        for box, pred, score in zip(bbox_pred, preds, scores):
                            y_box, x_box = box[:2]
                            h, w = box[2:4]
                            scale = float(w) / float(h)
                            
                            if scale > 0.9 and scale < 1.1:
                                
                                image_boxes.append([int(x_box + x_real), int(y_box + y_real), int(x_box + x_real + w), 
                                    int(y_box + y_real + h), int(pred), float(score)])                        
                                
        boxes = np.array(image_boxes)
        annos = non_max_suppression_by_distance(boxes, boxes[:, 5], 40)
        
        for anno in annos:       
                                
            annotations.append([str(slide_container.file.name), str(slide_container.file.parent.stem), "576255",
                                        slide_container.slide.level_dimensions[0][0], 
                                        slide_container.slide.level_dimensions[0][1],
                                        anno[0], anno[1], anno[2], anno[3], anno[4], anno[5]])
                            
                
            

        df = pd.DataFrame(annotations, columns=["file", "scanner", "image_nr", "width", "height" , "x1", "y1", "x2", "y2", "label", "score"])
        df.to_pickle("reg_results.p")

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [38:44<00:00, 581.18s/it]


In [50]:
df.head()

,file,scanner,image_nr,width,height,x1,y1,x2,y2,label,score
0,24_EIPH_576255 Berliner Blau.svs,NanoZoomerS210,576255,34200,32356,10978.0,7036.0,11107.0,7166.0,4.0,0.929032
1,24_EIPH_576255 Berliner Blau.svs,NanoZoomerS210,576255,34200,32356,30033.0,13813.0,30201.0,13981.0,2.0,0.926395
2,24_EIPH_576255 Berliner Blau.svs,NanoZoomerS210,576255,34200,32356,18278.0,4283.0,18394.0,4399.0,1.0,0.926342
3,24_EIPH_576255 Berliner Blau.svs,NanoZoomerS210,576255,34200,32356,25220.0,11058.0,25328.0,11167.0,1.0,0.925790
4,24_EIPH_576255 Berliner Blau.svs,NanoZoomerS210,576255,34200,32356,12767.0,22738.0,12885.0,22856.0,1.0,0.922600


In [43]:
slide_container.slide.level_dimensions[0]

(38400, 39424)

In [51]:
df["scanner"].unique()

array(['NanoZoomerS210'], dtype=object)

In [52]:
len(annotations)

33412

In [55]:
slide_container.file.parent.stem

'NanoZoomerS210'

In [57]:
[temp.file for temp in files] #.parent.stem

[WindowsPath('../Slides/Aperio/24_EIPH_576255 Berliner Blau.svs'),
 WindowsPath('../Slides/AxioScan/Z_BB_576255_1.tif'),
 WindowsPath('../Slides/NanoZoomer2.0HT/N2_BB_576255_1.ndpi'),
 WindowsPath('../Slides/NanoZoomerS210/N1_BB_576255_1.ndpi')]